<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/K_Nearest_Neighbors_Recommender/MovieLens_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> Collaborative Filtering con KNN en MovieLens</font></h1>

---

**Índice**

- 1. Introducción
- 2. Configuración del Entorno
- 3. Importación de Librerías
- 4. Carga y Preparación de Datos
- 5. Visualización de la Distribución Long Tail
- 6. Filtrado de Datos
- 7. Análisis de Usuarios Activos
- 8. Creación de la Matriz Usuario-Ítem
- 9. Enfoques de Filtrado Colaborativo
- 10. Estrategias de División de Datos
- 11. Construcción del Modelo KNN
- 12. Evaluación del Modelo
- 13. Análisis de Predicciones
- 14. Evaluación de Recomendaciones
- 15. Análisis de Diversidad
- 16. Conclusiones

## 1. Introducción


En este laboratorio, implementaremos un sistema de Filtrado Colaborativo utilizando el algoritmo K-Nearest Neighbors (KNN) sobre el dataset MovieLens 1M. Exploraremos los datos, visualizaremos distribuciones, aplicaremos filtrados y evaluaremos el rendimiento del modelo.

## 2. Configuración del Entorno


Primero, instalamos las librerías necesarias y descargamos el dataset.



In [1]:
# Instalar la librería scikit-surprise para algoritmos de filtrado colaborativo
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357278 sha256=9df4c389525375976e5b096d06af10f14b43481cf033dbd8c484cf8755358dbd
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
# Descargar el dataset MovieLens 1M
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  8052k      0 --:--:-- --:--:-- --:--:-- 8059k
Archive:  dataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
total 5800
drwxr-xr-x 1 root root    4096 Oct 29 01:23 .
drwxr-xr-x 1 root root    4096 Oct 29 01:18 ..
drwxr-xr-x 4 root root    4096 Oct 25 13:20 .config
-rw-r--r-- 1 root root 5917549 Oct 29 01:23 dataset.zip
drwxr-x--- 2 root root    4096 Jan 29  2016 ml-1m
drwxr-xr-x 1 root root    4096 Oct 25 13:20 sample_data


## 3. Importación de Librerías


Importamos las librerías que utilizaremos a lo largo del laboratorio.



In [3]:
import pandas as pd
import matplotlib.pyplot as plt


## 4. Carga y Preparación de Datos


### 4.1 Carga de Datos

Cargamos los datos de calificaciones y películas desde los archivos descargados.

In [4]:
# Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')


In [5]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


### 4.2 Unión de Datasets

Combinamos los datasets para incluir los títulos de las películas en las calificaciones.

In [7]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
user_item_rating.head(10)


,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4
33,1,Aladdin (1992),4
34,1,Mulan (1998),4
35,1,"Hunchback of Notre Dame, The (1996)",4
36,1,"Last Days of Disco, The (1998)",5
37,1,Cinderella (1950),5


# Preunta 2
## Se selecciona dos usuarios al azar que hayan valorado al menos 5 películas en comun

In [8]:
user_item_rating.columns

Index(['userId', 'title', 'rating'], dtype='object')

In [11]:
user_item_rating['title'].value_counts().head(5)

,count
title,
American Beauty (1999),3428
Star Wars: Episode IV - A New Hope (1977),2991
Star Wars: Episode V - The Empire Strikes Back (1980),2990
Star Wars: Episode VI - Return of the Jedi (1983),2883
Jurassic Park (1993),2672


In [17]:
user_item_rating.groupby(['title','userId']).count()

rating
title                  userId        
$1,000,000 Duck (1971) 216          1
                       494          1
                       714          1
                       869          1
                       1034         1
...                               ...
eXistenZ (1999)        5961         1
                       5964         1
                       6001         1
                       6016         1
                       6036         1

[1000209 rows x 1 columns]

In [19]:
# Obtencion de 2 usuarios con 5 perliculas en comun
df_pv = user_item_rating.pivot_table(index='userId', columns='title', values='rating')
df_pv

title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,3.0,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
'''
American Beauty (1999) 	3428
Star Wars: Episode IV - A New Hope (1977) 	2991
Star Wars: Episode V - The Empire Strikes Back (1980) 	2990
Star Wars: Episode VI - Return of the Jedi (1983) 	2883
Jurassic Park (1993)
'''
df_pv[['American Beauty (1999)', 'Star Wars: Episode IV - A New Hope (1977)',
      'Star Wars: Episode V - The Empire Strikes Back (1980)',
      'Star Wars: Episode VI - Return of the Jedi (1983)', 'Jurassic Park (1993)']].head(20)

title,American Beauty (1999),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Jurassic Park (1993)
userId,,,,,
1,NaN,4.0,NaN,NaN,NaN
2,4.0,NaN,5.0,4.0,5.0
3,4.0,5.0,4.0,4.0,4.0
4,NaN,5.0,2.0,3.0,4.0
5,4.0,NaN,NaN,NaN,NaN
6,1.0,NaN,NaN,3.0,NaN
7,NaN,NaN,5.0,NaN,4.0
8,5.0,NaN,NaN,4.0,5.0
9,4.0,NaN,NaN,4.0,4.0


In [30]:
# Al menos 2 usuarios con 5 peliculas en comun
# UserId: 3, 10, 15, 17, 19
df_filtrado = user_item_rating[(user_item_rating['userId'] == 3) | (user_item_rating['userId'] == 10)]
df_filtrado

,userId,title,rating
216,3,Young Guns (1988),5
218,3,Young Guns II (1990),4
217,3,"Silence of the Lambs, The (1991)",3
211,3,Star Wars: Episode V - The Empire Strikes Back...,4
214,3,"Princess Bride, The (1987)",5
...,...,...,...
929,10,"First Wives Club, The (1996)",5
928,10,Breakfast at Tiffany's (1961),5
926,10,Trading Places (1983),4
935,10,Gandhi (1982),4


## Calculo de similitud usando correlacion de Pearson

In [32]:
def calcular_similitud_pearson(df, usuario1_id, usuario2_id):
    # 1. Crear matriz pivote usuario-película
    user_movie_matrix = df.pivot_table(index='userId', columns='title', values='rating')


    usuario1_ratings = user_movie_matrix.loc[usuario1_id]
    usuario2_ratings = user_movie_matrix.loc[usuario2_id]

    peliculas_comunes = usuario1_ratings.notna() & usuario2_ratings.notna()

    ratings1 = usuario1_ratings[peliculas_comunes]
    ratings2 = usuario2_ratings[peliculas_comunes]

    pearson_corr = ratings1.corr(ratings2, method='pearson')

    comparacion = pd.DataFrame({
        'Película': ratings1.index,
        f'Ratings Usuario {usuario1_id}': ratings1.values,
        f'Ratings Usuario {usuario2_id}': ratings2.values
    })

    return pearson_corr, comparacion, len(ratings1)



In [34]:
# Ejemplo de uso
usuario1_id = 3
usuario2_id = 10

correlacion, comparacion, num_peliculas = calcular_similitud_pearson(user_item_rating, usuario1_id, usuario2_id)

# Mostrar resultados
print(f"Correlación de Pearson entre usuarios {usuario1_id} y {usuario2_id}: {correlacion:.3f}")
print(f"\nNúmero de películas en común: {num_peliculas}")
print("\nDetalle de calificaciones:")
print(comparacion)



Correlación de Pearson entre usuarios 3 y 10: 0.009

Número de películas en común: 31

Detalle de calificaciones:
                                             Película  Ratings Usuario 3  \
0                              American Beauty (1999)                4.0   
1                           Back to the Future (1985)                3.0   
2                         Being John Malkovich (1999)                3.0   
3                              Blazing Saddles (1974)                5.0   
4                                Bug's Life, A (1998)                5.0   
5           Butch Cassidy and the Sundance Kid (1969)                5.0   
6                             Crocodile Dundee (1986)                4.0   
7                           Dances with Wolves (1990)                4.0   
8                                  Dragonheart (1996)                4.0   
9                         Fish Called Wanda, A (1988)                5.0   
10                               Groundhog Day (19

In [35]:
# Análisis estadístico adicional
print("\nEstadísticas descriptivas:")
print(f"Usuario {usuario1_id}:")
print(f"Media de ratings: {comparacion[f'Ratings Usuario {usuario1_id}'].mean():.2f}")
print(f"Desviación estándar: {comparacion[f'Ratings Usuario {usuario1_id}'].std():.2f}")
print(f"\nUsuario {usuario2_id}:")
print(f"Media de ratings: {comparacion[f'Ratings Usuario {usuario2_id}'].mean():.2f}")
print(f"Desviación estándar: {comparacion[f'Ratings Usuario {usuario2_id}'].std():.2f}")


Estadísticas descriptivas:
Usuario 3:
Media de ratings: 3.97
Desviación estándar: 0.87

Usuario 10:
Media de ratings: 4.19
Desviación estándar: 0.79
